In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn import datasets
import matplotlib.pyplot as plt

from trb import objectives

from trb.learner import TRBLearner
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor

import warnings
warnings.filterwarnings('ignore')


In [2]:
n_samples = 500
n_outliers = 50

X, y, coef = datasets.make_regression(
    n_samples=n_samples,
    n_features=5,
    noise=10,
    coef=True,
    random_state=42,
)

# Add outlier data
np.random.seed(0)
X[:n_outliers] = 100 + 100 * np.random.normal(size=(n_outliers, 1))
y[:n_outliers] = -100 + 100 * np.random.normal(size=n_outliers)


# Add 5% strong outliers to the dataset.
X_outliers = X[:n_outliers, :]
y_outliers = y[:n_outliers]

X_inliers = X[n_outliers:, :]
y_inliers = y[n_outliers:]



In [3]:

def gbdt_reg_score(reg, X_test, y_test, n_estimators):
    loss = np.zeros((n_estimators,), dtype=np.float64)
    score = np.zeros((n_estimators,), dtype=np.float64)
    for i, y_pred in enumerate(reg.staged_predict(X_test)):
        loss[i] = reg.loss_(y_test, y_pred)
        score[i] = r2_score(y_test, y_pred)
    return loss, score



In [4]:
folder_path = "./reg_standard_result"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

dataset = 'huber'
objective = objectives.Huber()
n_estimators = 100

trb_best_loss_list = []
gbdt_best_loss_list = []

trb_best_r2_list = []
gbdt_best_r2_list = []

trb_best_nestimator_list = []
gbdt_best_nestimator_list = []

for i in range(5):
    np.random.seed(i)
    seed = np.random.randint(10000, size=1).item()
    X_train_in, X_test, y_train_in, y_test = train_test_split(
        X_inliers, y_inliers, test_size=0.2, random_state=seed)
    X_train = np.vstack((X_train_in, X_outliers))
    y_train = np.append(y_train_in, y_outliers)

    train_x, eval_x, train_y, eval_y = train_test_split(
        X_train, y_train, test_size=0.2, random_state=seed)

    # Trb grid search
    trb_best_nestimator = n_estimators
    trb_best_eval_loss = 1e15
    trb_best_alpha = 0.1
    trb_best_beta = 10
    trb_best_eta = 0
    alpha_list = [0.1, 0.5, 1, 5]
    beta_list = [0, 1, 10]
    eta_list = [0, 0.01, 0.1]
    for _alpha in alpha_list:
        for _eta in eta_list:
            trb_reg = TRBLearner(objective,
                                base_score=np.mean(train_y),  # 回归为0
                                n_estimators=n_estimators,
                                alpha=_alpha,
                                eta=_eta)
            trb_reg.fit(train_set=(train_x, train_y), eval_set=(eval_x, eval_y))
            trb_eval_loss = trb_reg.eval_loss_list
            if np.nanmin(trb_eval_loss) < trb_best_eval_loss:
                trb_best_nestimator = np.nanargmin(trb_eval_loss)+1
                trb_best_alpha = _alpha
                trb_best_eta = _eta
                trb_best_eval_loss = np.nanmin(trb_eval_loss)

    trb_reg = TRBLearner(objective,
                        base_score=np.mean(y_train),
                        n_estimators=trb_best_nestimator,
                        alpha=trb_best_alpha,
                        eta=trb_best_eta)
    trb_reg.fit(train_set=(X_train, y_train))
    y_trb_pred = trb_reg.predict(X_test)
    trb_r2 = r2_score(y_test, y_trb_pred) 
    trb_loss = objective.loss(y_test, y_trb_pred)
    trb_best_r2_list.append(trb_r2)   
    trb_best_nestimator_list.append(trb_best_nestimator)
    trb_best_loss_list.append(trb_loss)


    # Saving...
    current_path = os.path.join(
        folder_path, dataset, f'shuffle_{seed}', 'trb', 'evaluate')
    if not os.path.exists(current_path):
        os.makedirs(current_path)

    np.savez(os.path.join(current_path, "data.npz"), 
        loss = trb_loss,
        r2 = trb_r2,
        alpha = trb_best_alpha,
        eta = trb_best_eta,
        n_estiamtors = trb_best_nestimator,
    )


    # gbdt
    gbdt_best_nestimator = n_estimators
    gbdt_best_eval_loss = 1e15
    gbdt_best_lr = 0.1
    lr_list = [0.1, 0.5, 1]
    for lr in lr_list:
        gbdt_reg = GradientBoostingRegressor(loss='huber', n_estimators=n_estimators,
                                            learning_rate=lr,)
        gbdt_reg.fit(train_x, train_y)
        gbdt_eval_loss = gbdt_reg_score(gbdt_reg, eval_x, eval_y, n_estimators)[0]
        if np.nanmin(gbdt_eval_loss) < gbdt_best_eval_loss:
            gbdt_best_nestimator = np.nanargmin(gbdt_eval_loss)+1
            gbdt_best_lr = lr
            gbdt_best_eval_loss = np.nanmin(gbdt_eval_loss)

    gbdt_reg = GradientBoostingRegressor(loss='huber', n_estimators=gbdt_best_nestimator,
                                        learning_rate=gbdt_best_lr,)
    gbdt_reg.fit(X_train, y_train)
    y_gbdt_pred = gbdt_reg.predict(X_test)
    gbdt_r2 = r2_score(y_test, y_gbdt_pred)
    gbdt_loss = objective.loss(y_test, y_gbdt_pred)
    gbdt_best_r2_list.append(gbdt_r2)
    gbdt_best_nestimator_list.append(gbdt_best_nestimator)
    gbdt_best_loss_list.append(gbdt_loss)

    # Saving...
    current_path = os.path.join(
        folder_path, dataset, f'shuffle_{seed}', 'gbdt', 'evaluate')

    if not os.path.exists(current_path):
        os.makedirs(current_path)

    np.savez(os.path.join(current_path, "data.npz"), 
        loss = gbdt_loss,
        r2 = gbdt_r2,
        lr = gbdt_best_lr,
        n_estimators = gbdt_best_nestimator,
    )

    
print('TRB standard metric: mean={}, std={}'.format(np.mean(trb_best_r2_list),
                                    np.std(trb_best_r2_list)))
print('TRB standard loss: mean={}, std={}'.format(np.mean(trb_best_loss_list),
                                    np.std(trb_best_loss_list)))

print('GBDT metric: mean={}, std={}'.format(np.mean(gbdt_best_r2_list),
                                    np.std(gbdt_best_r2_list)))
print('GBDT loss: mean={}, std={}'.format(np.mean(gbdt_best_loss_list),
                                    np.std(gbdt_best_loss_list)))

TRB standard metric: mean=0.8848411850310198, std=0.02818208365748894
TRB standard loss: mean=26.456105452274777, std=3.9761579787974295
GBDT metric: mean=0.8226221789428994, std=0.024957652935373918
GBDT loss: mean=37.872863616655636, std=4.1529142154832295
